In [168]:
%pip install duckdb

Note: you may need to restart the kernel to use updated packages.


In [154]:
import duckdb

# basic SQL API

# connect to an in-memory temporary database
conn = duckdb.connect()

# if you want, you can create a cursor() like described in PEP 249 but its fully redundant
cursor = conn.cursor()

# run arbitrary SQL commands
conn.execute("CREATE TABLE test_table (i INTEGER, j STRING)")

In [155]:
conn.execute("INSERT INTO test_table VALUES (1, 'one')")

# we can use placeholders for parameters
conn.execute("INSERT INTO test_table VALUES (?, ?)", [2, 'two'])

In [156]:
# we can provide multiple sets of parameters to executemany()
conn.executemany("INSERT INTO test_table VALUES (?, ?)", [[3, 'three'], [4, 'four']])





In [157]:
# fetch as pandas data frame
print(conn.execute("SELECT * FROM test_table").fetchdf())



   i      j
0  1    one
1  2    two
2  3  three
3  4   four


In [158]:
# fetch as list of masked numpy arrays, cleaner when handling NULLs
print(conn.execute("SELECT * FROM test_table").fetchnumpy())


{'i': array([1, 2, 3, 4], dtype=int32), 'j': array(['one', 'two', 'three', 'four'], dtype=object)}


In [159]:
# we can query pandas data frames as if they were SQL views
# create a sample pandas data frame
import pandas as pd
test_df = pd.DataFrame.from_dict({"i":[1, 2, 3, 4], "j":["one", "two", "three", "four"]})


In [160]:
# make this data frame available as a view in duckdb
conn.register("test_df", test_df)
print(conn.execute("SELECT j FROM test_df WHERE i > 1").fetchdf())



       j
0    two
1  three
2   four


In [161]:
# relation API, programmatic querying. relations are lazily evaluated chains of relational operators

# create a "relation" from a pandas data frame with an existing connection
rel = conn.from_df(test_df)
print(rel)


┌───────┬─────────┐
│   i   │    j    │
│ int64 │ varchar │
├───────┼─────────┤
│     1 │ one     │
│     2 │ two     │
│     3 │ three   │
│     4 │ four    │
└───────┴─────────┘



In [162]:
# alternative shorthand, use a built-in default connection to create a relation from a pandas data frame
rel = duckdb.df(test_df)
print(rel)


┌───────┬─────────┐
│   i   │    j    │
│ int64 │ varchar │
├───────┼─────────┤
│     1 │ one     │
│     2 │ two     │
│     3 │ three   │
│     4 │ four    │
└───────┴─────────┘



In [163]:
# create a relation from a CSV file

# first create a CSV file from our pandas example
import tempfile, os
temp_file_name = os.path.join(tempfile.mkdtemp(), next(tempfile._get_candidate_names()))
test_df.to_csv(temp_file_name, index=False)


In [164]:
# now create a relation from it
rel = duckdb.from_csv_auto(temp_file_name)
print(rel)


┌───────┬─────────┐
│   i   │    j    │
│ int64 │ varchar │
├───────┼─────────┤
│     1 │ one     │
│     2 │ two     │
│     3 │ three   │
│     4 │ four    │
└───────┴─────────┘



In [165]:
# create a relation from an existing table
rel = conn.table("test_table")
print(rel)


┌───────┬─────────┐
│   i   │    j    │
│ int32 │ varchar │
├───────┼─────────┤
│     1 │ one     │
│     2 │ two     │
│     3 │ three   │
│     4 │ four    │
└───────┴─────────┘



In [166]:
# order the relation
print(rel.project('i + 1'))
print(rel.order('j'))


┌─────────┐
│ (i + 1) │
│  int32  │
├─────────┤
│       2 │
│       3 │
│       4 │
│       5 │
└─────────┘

┌───────┬─────────┐
│   i   │    j    │
│ int32 │ varchar │
├───────┼─────────┤
│     4 │ four    │
│     1 │ one     │
│     3 │ three   │
│     2 │ two     │
└───────┴─────────┘



In [167]:
# of course these things can be chained

print(rel.filter('i > 1').project('i + 1').order('j').limit(2))




BinderException: Binder Error: Referenced column "j" not found in FROM clause!
Candidate bindings: "test_table.(i + 1)"